# notre appli

Rendu le 16 févriller

In [1]:
import dash
import dash_bootstrap_components as dbc
from dash import Input, Output, State, dcc, html, Dash, dcc,html, callback
import pandas as pd 
import plotly.express as px
import Functions_dash as Fun
import Figure as fig
import pages as pag
from PIL import Image

In [2]:
app = dash.Dash(
    external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME, "assets/styles.css"],suppress_callback_exceptions=True
)

# Bar noir en haut de page
navbar = dbc.NavbarSimple(
    children=[
        dbc.Button("Menu", outline=True, color="secondary", className="mr-1", id="btn_sidebar"), # Bouton pour le menu
        dbc.Popover(
                    "CLiquez pour ranger/sortir le sidebar",
                    target='btn_sidebar',
                    trigger="hover",
                    placement = "bottom"
                 ), 
        # Select for merge
        dcc.Dropdown(
                    id='zone-data-filter',
                    options=[{"label":"Nationale","value":'all'},
                             {"label":"Régional","value":'reg'},
                             {"label":"Département","value":'dep'}
                            ],
                    value="all",
                    searchable=False,
                    clearable=False,
                    ),
        dbc.Popover([dbc.PopoverHeader("Choix de zone géographique"),
                     dbc.PopoverBody("Vous pouvez choisir la zone géographique à visualiser pour tout le dashboard")
                     ],
                    target='zone-data-filter',
                    trigger="hover",
                    placement = "right"
                    ),
        dcc.Dropdown(
                    id='zone-selection',
                    options=[],
                    value=None,
                    searchable=False,
                    clearable=False,
                    placeholder=""
                    )
    ],
    brand="",
    brand_href="#",
    color="dark",
    dark=True,
    fluid=True,
    links_left= True,
)


submenu_1 = [
    html.Li(
        # use Row and Col components to position the chevrons
        dbc.Row(
            [
                dbc.Col("Visualisations"),
                dbc.Col(
                    html.I(className="fas fa-chevron-right me-3"),
                    width="auto",
                ),
            ],
            className="my-1",
        ),
        style={"cursor": "pointer"},
        id="submenu-1",
    ),
    # we use the Collapse component to hide and reveal the navigation links
    dbc.Collapse(
        [
            dbc.NavLink("Evolution temporelle", href="/page-1/1"),
            dbc.NavLink("Caractéristiques des accidents", href="/page-1/2"),
        ],
        id="submenu-1-collapse",
    ),
]

submenu_2 = [
    html.Li(
        dbc.Row(
            [
                dbc.Col("Cartes interactives"),
                dbc.Col(
                    html.I(className="fas fa-chevron-right me-3"),
                    width="auto",
                ),
            ],
            className="my-1",
        ),
        style={"cursor": "pointer"},
        id="submenu-2",
    ),
    dbc.Collapse(
        [
            dbc.NavLink("Carte de la France", href="/page-map"),
            dbc.NavLink("Page 2.2", href="/page-2/2"),
        ],
        id="submenu-2-collapse",
    ),
]


sidebar = html.Div(
    [
        html.H2("DashBike", className="display-4"),
        html.Hr(),
        html.Img(src = Image.open("accident_bike.png"),style={"width": "60%",
                                                             "margin-bottom": '10%'}),
        html.P(
            "Vous trouverez ici les différentes pages du dashboard", className="lead"
        ),
        dbc.Nav(submenu_1 + submenu_2, vertical=True,pills=True),
        html.Img(src=Image.open("roue.png"),style={"width": "60%",
                                                  "margin-top": '140%'}),
    ],
    style=fig.SIDEBAR_STYLE,
    id="sidebar",
)


content = html.Div(id="page-content", style=fig.CONTENT_STYLE)

app.layout = html.Div(
    [
        dcc.Store(id='side_click'),
        dcc.Store(id='data-store', storage_type='session'),
        dcc.Location(id="url"),
        navbar,
        sidebar,
        content,
    ],
)



# Fonction qui appel toute les fonctions callback qui sont dans le fichier Fonction_dash.py
Fun.get_callbacks(app)

if __name__ == "__main__":
    app.run_server(debug=True,jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File c:\Users\andre\Documents\Cours\SSD\Data_vis\Figure.py:760, in bar(
    var='all',
    annee=2005,
    data=[{'Num_Acc': 200500002047, '_infos_commune.code_epci': 200071082, 'age': 69, 'age_group': 'Plus de 50 ans', 'agg': 'En agglomération', 'an': 2005, 'atm': 'Normale', 'catr': 'Voie Communales', 'choc': 'Côté droit', 'circ': 'Bidirectionnelle', ...}, {'Num_Acc': 201700002832, '_infos_commune.code_epci': 200071082, 'age': 67, 'age_group': 'Plus de 50 ans', 'agg': 'Hors agglomération', 'an': 2017, 'atm': 'Normale', 'catr': 'Route Départementale', 'choc': 'Aucun', 'circ': 'Bidirectionnelle', ...}, {'Num_Acc': 200500006797, '_infos_commune.code_epci': 200071363, 'age': 59, 'age_group': 'Plus de 50 ans', 'agg': 'En agglomération', 'an': 2005, 'atm': 'Normale', 'catr': 'Voie Communales', 'choc': 'Aucun', 'circ': 'Bidirectionnelle', ...

In [3]:
type(fig.data['reg'])

KeyError: 'reg'

In [4]:
fig.data.columns.values

array(['Num_Acc', 'date', 'an', 'mois', 'jour', 'hrmn', 'dep', 'com',
       'lat', 'long', 'agg', 'int', 'col', 'lum', 'atm', 'catr', 'circ',
       'nbv', 'prof', 'plan', 'lartpc', 'larrout', 'surf', 'infra',
       'situ', 'grav', 'sexe', 'age', 'trajet', 'secuexist', 'equipement',
       'obs', 'obsm', 'choc', 'manv', 'vehiculeid', 'typevehicules',
       'manoeuvehicules', 'numVehicules', '_infos_commune.code_epci',
       'region_name', 'age_group', 'com_name'], dtype=object)

In [4]:
fig.select_data(fig.data, {'dep' : ['01']})

,Num_Acc,date,an,mois,jour,hrmn,dep,com,lat,long,...,choc,manv,vehiculeid,typevehicules,manoeuvehicules,numVehicules,_infos_commune.code_epci,region_name,age_group,com_name
0,200500011098,2005-02-20,2005,février,dimanche,13:45,01,01364,46.396000,5.040000,...,Arrière droit,Traversant la chaussée,200500011098A01,7,1,1.0,200071751.0,Auvergne-Rhône-Alpes,Plus de 50 ans,Saint-Jean-sur-Reyssouze
1,201100006920,2011-05-21,2011,mai,samedi,18:15,01,01364,0.000000,0.000000,...,Avant,En s'insérant,201100006920A01,7,1,1.0,200071751.0,Auvergne-Rhône-Alpes,Moins de 18 ans,Saint-Jean-sur-Reyssouze
2,200500011121,2005-02-06,2005,février,dimanche,8:30,01,01052,0.000000,0.000000,...,Côté droit,Sans changement de direction,200500011121A01,7,19,1.0,200069193.0,Auvergne-Rhône-Alpes,Plus de 50 ans,Bouligneux
3,201300017798,2013-03-11,2013,mars,lundi,14:00,01,01052,46.025400,4.990550,...,Arrière droit,Sans changement de direction,201300017798B01,13,9,1.0,200069193.0,Auvergne-Rhône-Alpes,Plus de 50 ans,Bouligneux
4,200500013967,2005-03-29,2005,mars,mardi,17:20,01,01053,0.000000,0.000000,...,Avant,"Même sens, même file",200500013967B01,4,17,1.0,200071751.0,Auvergne-Rhône-Alpes,Plus de 50 ans,Bourg-en-Bresse
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,202100023950,2021-08-17,2021,août,mardi,18:30,01,01036,45.875706,5.664804,...,Avant,Inconnue,202100023950A01,10,1,1.0,200040350.0,Auvergne-Rhône-Alpes,Entre 30 et 50 ans,Belmont-Luthézieu
451,202100028215,2021-07-19,2021,juillet,lundi,12:45,01,01334,46.220821,4.923401,...,Arrière,Traversant la chaussée,202100028215B01,7,1,1.0,200070555.0,Auvergne-Rhône-Alpes,Plus de 50 ans,Saint-André-d'Huiriat
452,202100033934,2021-06-01,2021,juin,mardi,16:00,01,01193,45.640568,5.644087,...,Avant gauche,"Même sens, même file",202100033934B01,10,2,1.0,200040350.0,Auvergne-Rhône-Alpes,Plus de 50 ans,Izieu
453,202100040262,2021-05-12,2021,mai,mercredi,22:45,01,01145,46.068556,5.204322,...,Avant,Sans changement de direction,202100040262B01,2,1,1.0,200071751.0,Auvergne-Rhône-Alpes,Moins de 18 ans,Dompierre-sur-Veyle


In [10]:
fig.regions_code['nom_departement'].unique() # 'code_departement', 'nom_departement'

array(['Ain', 'Aisne', 'Allier', 'Alpes-de-Haute-Provence',
       'Hautes-Alpes', 'Alpes-Maritimes', 'Ardèche', 'Ardennes', 'Ariège',
       'Aube', 'Aude', 'Aveyron', 'Bouches-du-Rhône', 'Calvados',
       'Cantal', 'Charente', 'Charente-Maritime', 'Cher', 'Corrèze',
       "Côte-d'Or", "Côtes-d'Armor", 'Creuse', 'Dordogne', 'Doubs',
       'Drôme', 'Eure', 'Eure-et-Loir', 'Finistère', 'Corse-du-Sud',
       'Haute-Corse', 'Gard', 'Haute-Garonne', 'Gers', 'Gironde',
       'Hérault', 'Ille-et-Vilaine', 'Indre', 'Indre-et-Loire', 'Isère',
       'Jura', 'Landes', 'Loir-et-Cher', 'Loire', 'Haute-Loire',
       'Loire-Atlantique', 'Loiret', 'Lot', 'Lot-et-Garonne', 'Lozère',
       'Maine-et-Loire', 'Manche', 'Marne', 'Haute-Marne', 'Mayenne',
       'Meurthe-et-Moselle', 'Meuse', 'Morbihan', 'Moselle', 'Nièvre',
       'Nord', 'Oise', 'Orne', 'Pas-de-Calais', 'Puy-de-Dôme',
       'Pyrénées-Atlantiques', 'Hautes-Pyrénées', 'Pyrénées-Orientales',
       'Bas-Rhin', 'Haut-Rhin', 'Rhône',

In [4]:
dict((nb, f"{nb} {nom}") for nom, nb in zip(fig.regions_code['nom_departement'].unique(), fig.regions_code['code_departement'].unique()))['1']

'1 Ain'

In [25]:
dict((f"{nb} {nom}", nb) for nom, nb in zip(fig.regions_code['nom_departement'].unique(), fig.regions_code['code_departement'].unique()))

{'1 Ain': '1',
 '2 Aisne': '2',
 '3 Allier': '3',
 '4 Alpes-de-Haute-Provence': '4',
 '5 Hautes-Alpes': '5',
 '6 Alpes-Maritimes': '6',
 '7 Ardèche': '7',
 '8 Ardennes': '8',
 '9 Ariège': '9',
 '10 Aube': '10',
 '11 Aude': '11',
 '12 Aveyron': '12',
 '13 Bouches-du-Rhône': '13',
 '14 Calvados': '14',
 '15 Cantal': '15',
 '16 Charente': '16',
 '17 Charente-Maritime': '17',
 '18 Cher': '18',
 '19 Corrèze': '19',
 "21 Côte-d'Or": '21',
 "22 Côtes-d'Armor": '22',
 '23 Creuse': '23',
 '24 Dordogne': '24',
 '25 Doubs': '25',
 '26 Drôme': '26',
 '27 Eure': '27',
 '28 Eure-et-Loir': '28',
 '29 Finistère': '29',
 '2A Corse-du-Sud': '2A',
 '2B Haute-Corse': '2B',
 '30 Gard': '30',
 '31 Haute-Garonne': '31',
 '32 Gers': '32',
 '33 Gironde': '33',
 '34 Hérault': '34',
 '35 Ille-et-Vilaine': '35',
 '36 Indre': '36',
 '37 Indre-et-Loire': '37',
 '38 Isère': '38',
 '39 Jura': '39',
 '40 Landes': '40',
 '41 Loir-et-Cher': '41',
 '42 Loire': '42',
 '43 Haute-Loire': '43',
 '44 Loire-Atlantique': '44',


In [21]:
[f"{nb} {nom}" for nom, nb in zip(fig.regions_code['nom_departement'].unique(), fig.regions_code['code_departement'].unique())]

['1 Ain',
 '2 Aisne',
 '3 Allier',
 '4 Alpes-de-Haute-Provence',
 '5 Hautes-Alpes',
 '6 Alpes-Maritimes',
 '7 Ardèche',
 '8 Ardennes',
 '9 Ariège',
 '10 Aube',
 '11 Aude',
 '12 Aveyron',
 '13 Bouches-du-Rhône',
 '14 Calvados',
 '15 Cantal',
 '16 Charente',
 '17 Charente-Maritime',
 '18 Cher',
 '19 Corrèze',
 "21 Côte-d'Or",
 "22 Côtes-d'Armor",
 '23 Creuse',
 '24 Dordogne',
 '25 Doubs',
 '26 Drôme',
 '27 Eure',
 '28 Eure-et-Loir',
 '29 Finistère',
 '2A Corse-du-Sud',
 '2B Haute-Corse',
 '30 Gard',
 '31 Haute-Garonne',
 '32 Gers',
 '33 Gironde',
 '34 Hérault',
 '35 Ille-et-Vilaine',
 '36 Indre',
 '37 Indre-et-Loire',
 '38 Isère',
 '39 Jura',
 '40 Landes',
 '41 Loir-et-Cher',
 '42 Loire',
 '43 Haute-Loire',
 '44 Loire-Atlantique',
 '45 Loiret',
 '46 Lot',
 '47 Lot-et-Garonne',
 '48 Lozère',
 '49 Maine-et-Loire',
 '50 Manche',
 '51 Marne',
 '52 Haute-Marne',
 '53 Mayenne',
 '54 Meurthe-et-Moselle',
 '55 Meuse',
 '56 Morbihan',
 '57 Moselle',
 '58 Nièvre',
 '59 Nord',
 '60 Oise',
 '61 Orne

In [2]:
def adjust_dropdown_width(value, options):
    print(value)
    width = (
        # default width if you erased the value
        "100px"
        if value is None
        # this assumes that each letter in the label is 11px wide, you can
        # change this to your preference
        else f"{len(options[value]['label'])*11}px"
    )
    return [{'width': width}]

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '2A', '2B', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
       '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65',
       '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76',
       '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87',
       '88', '89', '90', '91', '92', '93', '94', '95', '971', '972',
       '973', '974', '976'], dtype=object)

In [2]:
fig.data_filter(fig.data, 'lat', x_select=[0,10])

,Unnamed: 0.1,Unnamed: 0,Num_Acc,date,an,mois,jour,hrmn,dep,com,...,_infos_commune.code_epci,code_postal,nom_commune_postal,latitude,longitude,nom_commune,nom_commune_complet,num_dep,dep_name,region_name
27128,27128,27128,200900002247,2009-01-13,2009,janvier,mardi,10:37,68,68066,...,246800726.0,NaN,NaN,NaN,NaN,non renseigné,NaN,68,Haut-Rhin,Grand Est
27129,27129,27129,200900002272,2009-01-27,2009,janvier,mardi,14:00,68,68298,...,246800395.0,NaN,NaN,NaN,NaN,non renseigné,NaN,68,Haut-Rhin,Grand Est
27130,27130,27130,200900002301,2009-01-03,2009,janvier,samedi,18:15,14,14366,...,200069532.0,NaN,NaN,NaN,NaN,non renseigné,NaN,14,Calvados,Normandie
27131,27131,27131,200900002301,2009-01-03,2009,janvier,samedi,18:15,14,14366,...,200069532.0,NaN,NaN,NaN,NaN,non renseigné,NaN,14,Calvados,Normandie
27229,27229,27229,200900003287,2009-01-22,2009,janvier,jeudi,10:15,30,30155,...,243000643.0,NaN,NaN,NaN,NaN,non renseigné,NaN,30,Gard,Occitanie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92703,92703,92703,202100037919,2021-05-28,2021,mai,vendredi,19:39,973,97307,...,249730045.0,NaN,NaN,NaN,NaN,non renseigné,NaN,973,Guyane,Guyane
93402,93402,93402,202100046842,2021-03-24,2021,mars,mercredi,15:07,973,97309,...,249730045.0,NaN,NaN,NaN,NaN,non renseigné,NaN,973,Guyane,Guyane
93568,93568,93568,202100049433,2021-02-23,2021,février,mardi,19:45,973,97353,...,249730037.0,97353.0,REGINA,3.940918,-52.519927,Régina,Régina,973,Guyane,Guyane
93717,93717,93717,202100050831,2021-02-20,2021,février,samedi,21:10,973,97305,...,249730045.0,NaN,NaN,NaN,NaN,non renseigné,NaN,973,Guyane,Guyane


In [10]:
fig.data[(fig.data['lat'] > [0,10][0]) & (fig.data['lat'] < [0,10][1])]

,Unnamed: 0.1,Unnamed: 0,Num_Acc,date,an,mois,jour,hrmn,dep,com,...,_infos_commune.code_epci,code_postal,nom_commune_postal,latitude,longitude,nom_commune,nom_commune_complet,num_dep,dep_name,region_name
27128,27128,27128,200900002247,2009-01-13,2009,janvier,mardi,10:37,68,68066,...,246800726.0,NaN,NaN,NaN,NaN,non renseigné,NaN,68,Haut-Rhin,Grand Est
27129,27129,27129,200900002272,2009-01-27,2009,janvier,mardi,14:00,68,68298,...,246800395.0,NaN,NaN,NaN,NaN,non renseigné,NaN,68,Haut-Rhin,Grand Est
27130,27130,27130,200900002301,2009-01-03,2009,janvier,samedi,18:15,14,14366,...,200069532.0,NaN,NaN,NaN,NaN,non renseigné,NaN,14,Calvados,Normandie
27131,27131,27131,200900002301,2009-01-03,2009,janvier,samedi,18:15,14,14366,...,200069532.0,NaN,NaN,NaN,NaN,non renseigné,NaN,14,Calvados,Normandie
27229,27229,27229,200900003287,2009-01-22,2009,janvier,jeudi,10:15,30,30155,...,243000643.0,NaN,NaN,NaN,NaN,non renseigné,NaN,30,Gard,Occitanie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92703,92703,92703,202100037919,2021-05-28,2021,mai,vendredi,19:39,973,97307,...,249730045.0,NaN,NaN,NaN,NaN,non renseigné,NaN,973,Guyane,Guyane
93402,93402,93402,202100046842,2021-03-24,2021,mars,mercredi,15:07,973,97309,...,249730045.0,NaN,NaN,NaN,NaN,non renseigné,NaN,973,Guyane,Guyane
93568,93568,93568,202100049433,2021-02-23,2021,février,mardi,19:45,973,97353,...,249730037.0,97353.0,REGINA,3.940918,-52.519927,Régina,Régina,973,Guyane,Guyane
93717,93717,93717,202100050831,2021-02-20,2021,février,samedi,21:10,973,97305,...,249730045.0,NaN,NaN,NaN,NaN,non renseigné,NaN,973,Guyane,Guyane


In [3]:
fig.fig_seri_reg(fig.data, [200, 800], [2010, 2015])

In [6]:
legend_groups = set()
for trace in fig.fig_seri_reg(fig.data)['data']:
    legend_groups.add(trace['legendgroup'])

In [7]:
print(legend_groups)

{"Provence-Alpes-Côte d'Azur", 'Corse', 'Grand Est', 'Pays de la Loire', 'Île-de-France', 'Guyane', 'Martinique', 'Mayotte', 'Bourgogne-Franche-Comté', 'Nouvelle-Aquitaine', 'Guadeloupe', 'Auvergne-Rhône-Alpes', 'Bretagne', 'La Réunion', 'Normandie', 'Occitanie', 'Centre-Val de Loire', 'Hauts-de-France'}


In [2]:
print(Fun.get_legend_groups(fig.fig_seri_reg(fig.data)))

{'Nouvelle-Aquitaine', 'Normandie', 'Mayotte', 'Guyane', 'Auvergne-Rhône-Alpes', "Provence-Alpes-Côte d'Azur", 'Occitanie', 'Bretagne', 'La Réunion', 'Corse', 'Guadeloupe', 'Hauts-de-France', 'Île-de-France', 'Bourgogne-Franche-Comté', 'Centre-Val de Loire', 'Pays de la Loire', 'Martinique', 'Grand Est'}


In [7]:
test = Fun.split_fig(fig.fig_seri_reg(fig.data))

In [9]:
test[1]

In [4]:
fig.data['an'].unique()

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021], dtype=int64)

In [6]:
fig.pie_age_grav('Blessé léger', 2005)['data'][0]['labels']

array(['Moins de 18 ans', 'Entre 18 et 30 ans', 'Entre 30 et 50 ans',
       'Plus de 50 ans'], dtype=object)

In [5]:
fig.fig_seri_reg(fig.data)['data'][0]['customdata'][0][1]

'Auvergne-Rhône-Alpes'

In [8]:
[trace for trace in fig.pie_age_grav('Blessé léger', 2005).select_traces(selector={'visible':'legendonly'})][0]

[]

In [15]:
fig.pie_age_grav('Blessé léger', 2005).to_json()

'{"data":[{"labels":["Moins de 18 ans","Entre 18 et 30 ans","Entre 30 et 50 ans","Plus de 50 ans"],"marker":{"colors":["#add8e6","#87ceeb","#6495ed","#4169e1"]},"pull":[0,0,0,0.2],"values":[881,830,1121,876],"type":"pie","hovertemplate":"Age de l\'usager: %{label}\\u003cbr\\u003eNombre d\'accidents: %{value}\\u003cbr\\u003ePourcentage: %{percent}","name":"Bless\\u00e9 l\\u00e9ger","visible":"legendonly"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0

In [5]:
fig.fig_seri_reg(fig.data)['data'][1]

Scatter({
    'customdata': array([[2005, 'Bourgogne-Franche-Comté', 528],
                         [2006, 'Bourgogne-Franche-Comté', 339],
                         [2007, 'Bourgogne-Franche-Comté', 434],
                         [2008, 'Bourgogne-Franche-Comté', 284],
                         [2009, 'Bourgogne-Franche-Comté', 291],
                         [2010, 'Bourgogne-Franche-Comté', 217],
                         [2011, 'Bourgogne-Franche-Comté', 228],
                         [2012, 'Bourgogne-Franche-Comté', 158],
                         [2013, 'Bourgogne-Franche-Comté', 159],
                         [2014, 'Bourgogne-Franche-Comté', 208],
                         [2015, 'Bourgogne-Franche-Comté', 241],
                         [2016, 'Bourgogne-Franche-Comté', 176],
                         [2017, 'Bourgogne-Franche-Comté', 172],
                         [2018, 'Bourgogne-Franche-Comté', 156],
                         [2019, 'Bourgogne-Franche-Comté', 253],
               

In [5]:
fig.fig_seri_reg(fig.data)['data'][0]['customdata']

array([[2005, 'Auvergne-Rhône-Alpes', 768],
       [2006, 'Auvergne-Rhône-Alpes', 762],
       [2007, 'Auvergne-Rhône-Alpes', 610],
       [2008, 'Auvergne-Rhône-Alpes', 594],
       [2009, 'Auvergne-Rhône-Alpes', 542],
       [2010, 'Auvergne-Rhône-Alpes', 521],
       [2011, 'Auvergne-Rhône-Alpes', 559],
       [2012, 'Auvergne-Rhône-Alpes', 568],
       [2013, 'Auvergne-Rhône-Alpes', 551],
       [2014, 'Auvergne-Rhône-Alpes', 583],
       [2015, 'Auvergne-Rhône-Alpes', 593],
       [2016, 'Auvergne-Rhône-Alpes', 568],
       [2017, 'Auvergne-Rhône-Alpes', 735],
       [2018, 'Auvergne-Rhône-Alpes', 538],
       [2019, 'Auvergne-Rhône-Alpes', 339],
       [2020, 'Auvergne-Rhône-Alpes', 332],
       [2021, 'Auvergne-Rhône-Alpes', 504]], dtype=object)

In [17]:
test = {
  "xaxis.autorange": "true",
  "xaxis.showspikes": "false",
  "yaxis.autorange": "true",
  "yaxis.showspikes": "false"
}

In [13]:
def relayoutData_transform(relayoutData):
    y_range = None
    x_range = None
        
    if 'autosize' in relayoutData:
        return None, None
        
    if 'xaxis.range[0]' in relayoutData or 'xaxis.range[1]' in relayoutData:
        x_range = [relayoutData['xaxis.range[0]'], relayoutData['xaxis.range[1]']]

    if 'yaxis.range[0]' in relayoutData or 'yaxis.range[1]' in relayoutData:
        y_range = [relayoutData['yaxis.range[0]'], relayoutData['yaxis.range[1]']]

    if "xaxis.autorange" in relayoutData:
         x_range = None

    if "yaxis.autorange" in relayoutData:
        y_range = None
        
    return x_range, y_range

In [18]:
relayoutData_transform(test)

(None, None)

In [24]:
fig.data[(fig.data['lat'] < -20) | (fig.data['lat'] > 0)]

,Unnamed: 0.1,Unnamed: 0,Num_Acc,date,an,mois,jour,hrmn,dep,com,...,_infos_commune.code_epci,code_postal,nom_commune_postal,latitude,longitude,nom_commune,nom_commune_complet,num_dep,dep_name,region_name
0,0,0,200500000030,2005-01-13,2005,janvier,jeudi,19:45,62,62331,...,200033579.0,NaN,NaN,NaN,NaN,non renseigné,NaN,62,Pas-de-Calais,Hauts-de-France
3,3,3,200500000093,2005-01-03,2005,janvier,lundi,13:30,02,02810,...,200071991.0,NaN,NaN,NaN,NaN,non renseigné,NaN,02,Aisne,Hauts-de-France
6,6,6,200500000260,2005-01-04,2005,janvier,mardi,16:45,22,22143,...,200068989.0,NaN,NaN,NaN,NaN,non renseigné,NaN,22,Côtes-d'Armor,Bretagne
10,10,10,200500000291,2005-01-24,2005,janvier,lundi,17:30,56,56155,...,244400610.0,NaN,NaN,NaN,NaN,non renseigné,NaN,56,Morbihan,Bretagne
14,14,14,200500000329,2005-01-04,2005,janvier,mardi,19:00,50,50213,...,NaN,NaN,NaN,NaN,NaN,non renseigné,NaN,50,Manche,Normandie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,94044,94044,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,POEY D OLORON,43.246755,-0.663214,Poey-d'Oloron,Poey-d'Oloron,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine
94045,94045,94045,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,PRECILHON,43.195778,-0.572974,Précilhon,Précilhon,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine
94046,94046,94046,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,ST GOIN,43.231201,-0.725099,Saint-Goin,Saint-Goin,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine
94047,94047,94047,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,SAUCEDE,43.265674,-0.669442,Saucède,Saucède,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine


In [19]:
fig.data.columns.values

array(['Unnamed: 0.1', 'Unnamed: 0', 'Num_Acc', 'date', 'an', 'mois',
       'jour', 'hrmn', 'dep', 'com', 'lat', 'long', 'agg', 'int', 'col',
       'lum', 'atm', 'catr', 'circ', 'nbv', 'prof', 'plan', 'lartpc',
       'larrout', 'surf', 'infra', 'situ', 'grav', 'sexe', 'age',
       'trajet', 'secuexist', 'equipement', 'obs', 'obsm', 'choc', 'manv',
       'vehiculeid', 'typevehicules', 'manoeuvehicules', 'numVehicules',
       '_infos_commune.code_epci', 'code_postal', 'nom_commune_postal',
       'latitude', 'longitude', 'nom_commune', 'nom_commune_complet',
       'num_dep', 'dep_name', 'region_name'], dtype=object)

## Temp storage
### Number bar children

In [ ]:
dbc.Row(
                        [
                            dbc.Col(fig.nb_total, width=3),
                            dbc.Col(fig.nb_mort, width=3),
                            dbc.Col(fig.nb_hospital, width=3),
                            dbc.Col("Info 4", width=3),
                        ],
                    ),

In [6]:
fig.data[data]

,Country Name,Indicator Name,Year,Value
0,Arab World,"Agriculture, value added (% of GDP)",1962,NaN
1,Arab World,CO2 emissions (metric tons per capita),1962,0.760996
2,Arab World,Domestic credit provided by financial sector (...,1962,18.168690
3,Arab World,Electric power consumption (kWh per capita),1962,NaN
4,Arab World,Energy use (kg of oil equivalent per capita),1962,NaN
...,...,...,...,...
36955,Zimbabwe,"Industry, value added (% of GDP)",2007,33.074953
36956,Zimbabwe,"Inflation, GDP deflator (annual %)",2007,0.894887
36957,Zimbabwe,"Life expectancy at birth, total (years)",2007,44.177756
36958,Zimbabwe,Population density (people per sq. km of land ...,2007,34.374559


In [3]:
fig.select_data(fig.data, {'dep':['1']})

,Num_Acc,date,an,mois,jour,hrmn,dep,com,lat,long,...,choc,manv,vehiculeid,typevehicules,manoeuvehicules,numVehicules,_infos_commune.code_epci,region_name,age_group,com_name


In [4]:
from dash import Dash, dcc, html, Input, Output, callback
from plotly.express import data
import pandas as pd

df =  data.medals_long()

all_options = {
                'all': [],
                'reg': [{'label': '84 Auvergne-Rhône-Alpes','value': 84},
                        {'label': '32 Hauts-de-France','value': 32},
                        {'label': '93 Provence-Alpes-Côte d\'Azur','value': 93},
                        {'label': '75 Nouvelle-Aquitaine','value': 75},
                        {'label': '24 Centre-Val de Loire','value': 24},
                        {'label': '27 Bourgogne-Franche-Comté','value': 27},
                        {'label': '28 Normandie','value': 28},
                        {'label': '53 Bretagne','value': 53},
                        {'label': '76 Occitanie','value': 76},
                        {'label': '52 Pays de la Loire','value': 52},
                        {'label': '44 Grand Est','value': 44},
                        {'label': '11 Île-de-France','value': 11},
                        {'label': '94 Corse','value': 94}
                        ],
                'dep': dict((nb, f"{nb} {nom}") for nom, nb in zip(fig.regions_code['nom_departement'].unique(), fig.regions_code['code_departement'].unique()))
            }

app = Dash(__name__)
app.layout = html.Div([
    dcc.Dropdown(options=all_options['dep'], id='pandas-dropdown-2'),
    html.Div(id='pandas-output-container-2',style={'color':'white'})
])


@callback(
    Output('pandas-output-container-2', 'children'),
    Input('pandas-dropdown-2', 'value')
)
def update_output(value):
    return f'You have selected {value}'


if __name__ == '__main__':
    app.run(debug=True)

In [8]:
code = '1'
code = '0' + code
print(code)

01


In [10]:

dcc.Graph(figure=fig.fig_seri_reg(fig.data))[0]

Graph(figure=Figure({
    'data': [{'customdata': array([[2005, 'Auvergne-Rhône-Alpes', 768],
                                   [2006, 'Auvergne-Rhône-Alpes', 762],
                                   [2007, 'Auvergne-Rhône-Alpes', 610],
                                   [2008, 'Auvergne-Rhône-Alpes', 594],
                                   [2009, 'Auvergne-Rhône-Alpes', 542],
                                   [2010, 'Auvergne-Rhône-Alpes', 521],
                                   [2011, 'Auvergne-Rhône-Alpes', 559],
                                   [2012, 'Auvergne-Rhône-Alpes', 568],
                                   [2013, 'Auvergne-Rhône-Alpes', 551],
                                   [2014, 'Auvergne-Rhône-Alpes', 583],
                                   [2015, 'Auvergne-Rhône-Alpes', 593],
                                   [2016, 'Auvergne-Rhône-Alpes', 568],
                                   [2017, 'Auvergne-Rhône-Alpes', 735],
                                   [2018, 

In [11]:
fig.data

,Unnamed: 0.1,Unnamed: 0,Num_Acc,date,an,mois,jour,hrmn,dep,com,...,_infos_commune.code_epci,code_postal,nom_commune_postal,latitude,longitude,nom_commune,nom_commune_complet,num_dep,dep_name,region_name
0,0,0,200500000030,2005-01-13,2005,janvier,jeudi,19:45,62,62331,...,200033579.0,NaN,NaN,NaN,NaN,non renseigné,NaN,62,Pas-de-Calais,Hauts-de-France
1,1,1,200500000034,2005-01-19,2005,janvier,mercredi,10:45,62,62022,...,200018083.0,NaN,NaN,NaN,NaN,non renseigné,NaN,62,Pas-de-Calais,Hauts-de-France
2,2,2,200500000078,2005-01-26,2005,janvier,mercredi,13:15,02,02173,...,200071785.0,NaN,NaN,NaN,NaN,non renseigné,NaN,02,Aisne,Hauts-de-France
3,3,3,200500000093,2005-01-03,2005,janvier,lundi,13:30,02,02810,...,200071991.0,NaN,NaN,NaN,NaN,non renseigné,NaN,02,Aisne,Hauts-de-France
4,4,4,200500000170,2005-01-29,2005,janvier,samedi,18:30,76,76196,...,200084952.0,NaN,NaN,NaN,NaN,non renseigné,NaN,76,Seine-Maritime,Normandie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,94044,94044,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,POEY D OLORON,43.246755,-0.663214,Poey-d'Oloron,Poey-d'Oloron,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine
94045,94045,94045,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,PRECILHON,43.195778,-0.572974,Précilhon,Précilhon,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine
94046,94046,94046,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,ST GOIN,43.231201,-0.725099,Saint-Goin,Saint-Goin,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine
94047,94047,94047,202100056508,2021-01-01,2021,janvier,vendredi,12:20,64,64400,...,246401756.0,64400.0,SAUCEDE,43.265674,-0.669442,Saucède,Saucède,64,Pyrénées-Atlantiques,Nouvelle-Aquitaine
